# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f048c661ee0>
├── 'a' --> tensor([[-1.3243,  0.4432, -0.2395],
│                   [ 0.8267,  0.6192, -0.3768]])
└── 'x' --> <FastTreeValue 0x7f048c661b50>
    └── 'c' --> tensor([[-1.7263,  1.7751,  0.1257, -1.5288],
                        [-0.0420, -0.4768,  0.2027,  0.2390],
                        [ 0.4948, -0.4152,  0.2767,  0.0952]])

In [4]:
t.a

tensor([[-1.3243,  0.4432, -0.2395],
        [ 0.8267,  0.6192, -0.3768]])

In [5]:
%timeit t.a

61.6 ns ± 0.059 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f048c661ee0>
├── 'a' --> tensor([[-0.2160,  2.3157,  0.0744],
│                   [ 0.0662, -1.0390,  2.0234]])
└── 'x' --> <FastTreeValue 0x7f048c661b50>
    └── 'c' --> tensor([[-1.7263,  1.7751,  0.1257, -1.5288],
                        [-0.0420, -0.4768,  0.2027,  0.2390],
                        [ 0.4948, -0.4152,  0.2767,  0.0952]])

In [7]:
%timeit t.a = new_value

66.6 ns ± 0.0232 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-1.3243,  0.4432, -0.2395],
               [ 0.8267,  0.6192, -0.3768]]),
    x: Batch(
           c: tensor([[-1.7263,  1.7751,  0.1257, -1.5288],
                      [-0.0420, -0.4768,  0.2027,  0.2390],
                      [ 0.4948, -0.4152,  0.2767,  0.0952]]),
       ),
)

In [10]:
b.a

tensor([[-1.3243,  0.4432, -0.2395],
        [ 0.8267,  0.6192, -0.3768]])

In [11]:
%timeit b.a

53.4 ns ± 0.0269 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 1.0812, -0.3242,  0.4402],
               [ 0.0399,  0.3113, -0.4708]]),
    x: Batch(
           c: tensor([[-1.7263,  1.7751,  0.1257, -1.5288],
                      [-0.0420, -0.4768,  0.2027,  0.2390],
                      [ 0.4948, -0.4152,  0.2767,  0.0952]]),
       ),
)

In [13]:
%timeit b.a = new_value

479 ns ± 0.15 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

871 ns ± 0.432 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.4 µs ± 26 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

140 µs ± 1.38 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

140 µs ± 179 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f03e6f1cf10>
├── 'a' --> tensor([[[-1.3243,  0.4432, -0.2395],
│                    [ 0.8267,  0.6192, -0.3768]],
│           
│                   [[-1.3243,  0.4432, -0.2395],
│                    [ 0.8267,  0.6192, -0.3768]],
│           
│                   [[-1.3243,  0.4432, -0.2395],
│                    [ 0.8267,  0.6192, -0.3768]],
│           
│                   [[-1.3243,  0.4432, -0.2395],
│                    [ 0.8267,  0.6192, -0.3768]],
│           
│                   [[-1.3243,  0.4432, -0.2395],
│                    [ 0.8267,  0.6192, -0.3768]],
│           
│                   [[-1.3243,  0.4432, -0.2395],
│                    [ 0.8267,  0.6192, -0.3768]],
│           
│                   [[-1.3243,  0.4432, -0.2395],
│                    [ 0.8267,  0.6192, -0.3768]],
│           
│                   [[-1.3243,  0.4432, -0.2395],
│                    [ 0.8267,  0.6192, -0.3768]]])
└── 'x' --> <FastTreeValue 0x7f03e6f1c7c0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

31.6 µs ± 88.9 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f03e03db3a0>
├── 'a' --> tensor([[-1.3243,  0.4432, -0.2395],
│                   [ 0.8267,  0.6192, -0.3768],
│                   [-1.3243,  0.4432, -0.2395],
│                   [ 0.8267,  0.6192, -0.3768],
│                   [-1.3243,  0.4432, -0.2395],
│                   [ 0.8267,  0.6192, -0.3768],
│                   [-1.3243,  0.4432, -0.2395],
│                   [ 0.8267,  0.6192, -0.3768],
│                   [-1.3243,  0.4432, -0.2395],
│                   [ 0.8267,  0.6192, -0.3768],
│                   [-1.3243,  0.4432, -0.2395],
│                   [ 0.8267,  0.6192, -0.3768],
│                   [-1.3243,  0.4432, -0.2395],
│                   [ 0.8267,  0.6192, -0.3768],
│                   [-1.3243,  0.4432, -0.2395],
│                   [ 0.8267,  0.6192, -0.3768]])
└── 'x' --> <FastTreeValue 0x7f048c625190>
    └── 'c' --> tensor([[-1.7263,  1.7751,  0.1257, -1.5288],
                        [-0.0420, -0.4768,  0.2027,  0.2390],
                 

In [23]:
%timeit t_cat(trees)

29.8 µs ± 77.3 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

59.9 µs ± 133 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[-1.7263,  1.7751,  0.1257, -1.5288],
                       [-0.0420, -0.4768,  0.2027,  0.2390],
                       [ 0.4948, -0.4152,  0.2767,  0.0952]],
              
                      [[-1.7263,  1.7751,  0.1257, -1.5288],
                       [-0.0420, -0.4768,  0.2027,  0.2390],
                       [ 0.4948, -0.4152,  0.2767,  0.0952]],
              
                      [[-1.7263,  1.7751,  0.1257, -1.5288],
                       [-0.0420, -0.4768,  0.2027,  0.2390],
                       [ 0.4948, -0.4152,  0.2767,  0.0952]],
              
                      [[-1.7263,  1.7751,  0.1257, -1.5288],
                       [-0.0420, -0.4768,  0.2027,  0.2390],
                       [ 0.4948, -0.4152,  0.2767,  0.0952]],
              
                      [[-1.7263,  1.7751,  0.1257, -1.5288],
                       [-0.0420, -0.4768,  0.2027,  0.2390],
                       [ 0.4948, -0.4152,  0.2767,  0.0952]],

In [26]:
%timeit Batch.stack(batches)

78.3 µs ± 104 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[-1.7263,  1.7751,  0.1257, -1.5288],
                      [-0.0420, -0.4768,  0.2027,  0.2390],
                      [ 0.4948, -0.4152,  0.2767,  0.0952],
                      [-1.7263,  1.7751,  0.1257, -1.5288],
                      [-0.0420, -0.4768,  0.2027,  0.2390],
                      [ 0.4948, -0.4152,  0.2767,  0.0952],
                      [-1.7263,  1.7751,  0.1257, -1.5288],
                      [-0.0420, -0.4768,  0.2027,  0.2390],
                      [ 0.4948, -0.4152,  0.2767,  0.0952],
                      [-1.7263,  1.7751,  0.1257, -1.5288],
                      [-0.0420, -0.4768,  0.2027,  0.2390],
                      [ 0.4948, -0.4152,  0.2767,  0.0952],
                      [-1.7263,  1.7751,  0.1257, -1.5288],
                      [-0.0420, -0.4768,  0.2027,  0.2390],
                      [ 0.4948, -0.4152,  0.2767,  0.0952],
                      [-1.7263,  1.7751,  0.1257, -1.5288],
                   

In [28]:
%timeit Batch.cat(batches)

143 µs ± 349 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

310 µs ± 447 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
